<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_27_Multitenant_Content_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to leverage multitenancy in Graphlit. We will show how to ingest content at 'project-scope' and 'tenant-scope', and how the data partitioning works when querying content.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [25]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [26]:
import os
import uuid
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

# Configure unique tenant identifiers

tenant1_id = str(uuid.uuid4())
tenant2_id = str(uuid.uuid4())


Define Graphlit helper functions

In [27]:
from typing import List, Optional

async def ingest_uri(uri: str):
    if graphlit.client is None:
        return;

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def query_contents():
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents()

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)


Execute Graphlit example

In [28]:
from IPython.display import display, Markdown

# Remove any existing contents; only needed for notebook example
await delete_all_contents()

print('Deleted all contents.')

# Tenants will inherit any content from the project-scope, allowing for common content to be injected into all tenant's knowledge bases

# Assign unique tenant (owner) identifier and refresh the Graphlit client
graphlit.owner_id = tenant1_id
graphlit.refresh_client()

print()
print('Ingesting image into Tenant 1-scope')

await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/Images/Chicago/20170617_145925.jpg")

# We should return the one Tenant 1-scoped image
contents = await query_contents()

if contents is not None:
    display(Markdown(f'### Tenant 1-scoped content'))
    for content in contents:
        if content is not None:
            print(f"[{content.id}]: {content.name}")

display(Markdown("We have now uploaded an image into Tenant 1.  When we query Tenant 1's contents, we should return back that one image."))

# Assign unique tenant (owner) identifier and refresh the Graphlit client
graphlit.owner_id = tenant2_id
graphlit.refresh_client()

print()
print('Ingesting image into Tenant 2-scope')

await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/Images/Chicago/20170816_214947.jpg")

# We should return the one Tenant 2-scoped image
contents = await query_contents()

if contents is not None:
    display(Markdown(f'### Tenant 2-scoped content'))
    for content in contents:
        if content is not None:
            print(f"[{content.id}]: {content.name}")

display(Markdown("We have now uploaded an image into Tenant 2.  When we query Tenant 2's contents, we should return back that one image."))

# Reset tenant identifier (to project-scope) and refresh the Graphlit client
graphlit.owner_id = None
graphlit.refresh_client()

print()
print('Ingesting image into Project-scope')

await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/Images/Chicago/20170601_081526.jpg")

# We should return the one project-scoped image
contents = await query_contents()

if contents is not None:
    display(Markdown(f'### Project-scoped content'))
    for content in contents:
        if content is not None:
            print(f"[{content.id}]: {content.name}")

display(Markdown("We have now uploaded an image into the project.  When we query the project's contents, we should get back three images - one from the project, and one from each tenant. The project has access to all tenant data, but not vice versa. Tenant's only have access to their own data, and any data inherited from the project."))

# Assign unique tenant (owner) identifier and refresh the Graphlit client
graphlit.owner_id = tenant1_id
graphlit.refresh_client()

print()
print('Ingesting image into Tenant 1-scope')

await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/Images/Chicago/20170601_082315.jpg")

# We should return the one Tenant 1-scoped image, plus the one project-scoped image
contents = await query_contents()

if contents is not None:
    display(Markdown(f'### Tenant 1-scoped content (including Project-scoped Content)'))
    for content in contents:
        if content is not None:
            print(f"[{content.id}]: {content.name}")

display(Markdown("We have now uploaded another image into Tenant 1, but we also already uploaded an image into the project.  When we query Tenant 1's contents, we should return back that three images - two from the tenant, and one inherited from the project."))

# Assign unique tenant (owner) identifier and refresh the Graphlit client
graphlit.owner_id = tenant2_id
graphlit.refresh_client()

print()
print('Ingesting image into Tenant 2-scope')

await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/Images/Chicago/20170601_213903.jpg")

# We should return the one Tenant 2-scoped image, plus the one project-scoped image
contents = await query_contents()

if contents is not None:
    display(Markdown(f'### Tenant 2-scoped content (including Project-scoped Content)'))
    for content in contents:
        if content is not None:
            print(f"[{content.id}]: {content.name}")

display(Markdown("We have now uploaded another image into Tenant 2, but we also already uploaded an image into the project.  When we query Tenant 2's contents, we should return back that three images - two from the tenant, and one inherited from the project."))

# Reset tenant identifier (to project-scope) and refresh the Graphlit client
graphlit.owner_id = None
graphlit.refresh_client()

# We should return the one project-scoped image, plus the Tenant 1 and Tenant 2-scoped images
contents = await query_contents()

if contents is not None:
    display(Markdown(f'### Project-scoped content (including all Tenant-scoped Content)'))
    for content in contents:
        if content is not None:
            print(f"[{content.id}]: {content.name}")

display(Markdown("When we query the project's contents, we get all the data back from the project including any tenant data.  We should now have 5 images total."))

# We must clean up each tenant separately, in its own scope; deleting contents at project-scope won't delete contents at tenant-scope

graphlit.owner_id = tenant1_id
graphlit.refresh_client()

await delete_all_contents()

graphlit.owner_id = tenant2_id
graphlit.refresh_client()

await delete_all_contents()

# Reset tenant identifier (to project-scope) and refresh the Graphlit client
graphlit.owner_id = None
graphlit.refresh_client()

# We should return the one project-scoped image
contents = await query_contents()

if contents is not None:
    display(Markdown(f'### Project-scoped content'))
    for content in contents:
        if content is not None:
            print(f"[{content.id}]: {content.name}")

display(Markdown("At the end, when we query the project's contents, we should get back the one image we uploaded into the project. All tenant data has already been deleted."))


Deleted all contents.

Ingesting image into Tenant 1-scope


### Tenant 1-scoped content

[2795fbc3-99d4-4bb5-bbc5-3a960321abff]: 20170617_145925.jpg


We have now uploaded an image into Tenant 1.  When we query Tenant 1's contents, we should return back that one image.


Ingesting image into Tenant 2-scope


...

[46bdbe70-dd8a-48bc-b97e-961593af0388]: 20170816_214947.jpg
[2795fbc3-99d4-4bb5-bbc5-3a960321abff]: 20170617_145925.jpg
[34b472b5-3ebb-483d-8090-5e6b7cfdc994]: 20170601_213903.jpg
[f2ac4d8a-064f-4fe4-abf4-7c98c6db51c3]: 20170601_082315.jpg
[0357e32f-de24-4370-b8b1-913d566e7d00]: 20170601_081526.jpg


When we query the project's contents, we get all the data back from the project including any tenant data.  We should now have 5 images total.

### Project-scoped content

[0357e32f-de24-4370-b8b1-913d566e7d00]: 20170601_081526.jpg


At the end, when we query the project's contents, we should get back the one image we uploaded into the project. All tenant data has already been deleted.